# Noteboook to define household clustering according to demographics

In this notebook the households are grouped based on three demographic statistics: 'Grossregion01', 'Haushaltstyp14', and 'Einkommensklasse08_151617'
Because of privacy guidelines no groups can be made with fewer than 150 households, therefore only combinations of two out of three of the above catergories are used to define groups. E.g. Grossregion and Einkommenklasse, or Haushalttyp and Einkommensklasse. Subgroups that are smaller than 150 households are aggregated into 'rest groups' for the respective combination of categories. 



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, product

In [3]:
data_description = pd.read_excel('/Users/ajakobs/Documents/data/HABE/131_EBM 2015-2017/HABE151617_191111UOe/HABE151617_Datenbeschreibung_191111UOe.xlsx', sheet_name='Modalitätsbezeichnungen', skiprows=10, usecols=[3,4,7], skipfooter=5)
data_description.drop(0, inplace=True)
data_description

,Variablenname,Modalität,Bezeichnung
1,Grossregion01,1.0,Genferseeregion
2,Grossregion01,2.0,Espace Mittelland
3,Grossregion01,3.0,Nordwestschweiz
4,Grossregion01,4.0,Zürich
5,Grossregion01,5.0,Ostschweiz
...,...,...,...
154,Erwerbsstatus05,13.0,unselbständigerwerbend
155,Erwerbsstatus05,32.0,in Ausbildung
156,Erwerbsstatus05,34.0,Rentner/in
157,Erwerbsstatus05,39.0,"andere Situation (erwerbslos, Hausfrau/Hausman..."


In [4]:
hh_data = pd.read_csv('/Users/ajakobs/Documents/data/HABE/131_EBM 2015-2017/HABE151617_191111UOe/HABE151617_Standard_191111UOe.txt', delimiter='\t', index_col=0)
hh_data


,Strate01,SRH03_151617,Gewicht10_151617,E10,E11,E12,E15,E20,E21,E22,...,Grossregion01,Sprachregion98,Kanton08,AltersklasseRefP08,Einkommensklasse08_151617,Haushaltstyp14,HaushaltstypAnzahlKinder14,HaushaltstypAlter14,HaushaltstypGrob14,HaushaltstypEinkommen14_151617
HaushaltID,,,,,,,,,,,,,,,,,,,,,
360101,1,676802.50000,511.219688,8274.925833,5509.146667,2765.779167,0.291667,0.000000,0.0,0.000000,...,1,2,22,1,3,210,900,202,200,212
360102,1,676802.50000,360.257688,5212.875000,5212.875000,0.000000,0.141667,0.000000,0.0,0.000000,...,1,2,22,1,2,110,900,101,100,112
360111,1,676802.50000,550.323306,0.000000,0.000000,0.000000,0.095833,605.475000,0.0,0.000000,...,1,2,99,1,1,210,900,201,200,211
360122,1,676802.50000,413.061166,5895.979167,5895.979167,0.000000,0.000000,1090.000000,0.0,0.000000,...,1,2,25,1,2,300,900,900,900,900
360125,1,676802.50000,550.652917,7913.543414,7913.543414,0.000000,21.920833,3265.766667,1880.0,1385.766667,...,1,2,22,4,4,900,900,900,900,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431723,7,159906.58333,872.239377,8105.142473,0.000000,8105.142473,8.333333,0.000000,0.0,0.000000,...,7,3,21,4,3,900,900,900,900,900
431730,7,159906.58333,172.619969,8972.750000,8972.750000,0.000000,55.833333,400.000000,0.0,0.000000,...,7,3,21,3,3,400,401,403,900,402
431738,7,159906.58333,100.951392,0.000000,0.000000,0.000000,0.153354,3395.500000,0.0,0.000000,...,7,3,21,1,2,300,900,900,900,900


In [5]:
hh_data.columns

Index(['Strate01', 'SRH03_151617', 'Gewicht10_151617', 'E10', 'E11', 'E12',
       'E15', 'E20', 'E21', 'E22', 'E23', 'E25', 'E70', 'A30m', 'A31m', 'A32m',
       'A33m', 'A35m', 'A40m', 'A41m', 'A42m', 'A43m', 'A44m', 'A50m', 'A51m',
       'A52m', 'A53m', 'A56m', 'A57m', 'A58m', 'A61m', 'A62m', 'A63m', 'A66m',
       'A69m', 'Primaereinkommen08', 'Bruttoeinkommen08',
       'VerfuegbaresEinkommen08', 'Sparbetrag08', 'AnzahlPersonen98',
       'AnzahlSelbstaendiger05', 'AnzahlUnselbstaendiger05', 'AnzahlRentner05',
       'AnzahlAusbildung05', 'AnzahlAndere05', 'AnzahlKinder05',
       'Anzahl0004Personen08', 'Anzahl0514Personen08', 'Anzahl1524Personen08',
       'Anzahl2534Personen08', 'Anzahl3544Personen08', 'Anzahl4554Personen08',
       'Anzahl5564Personen08', 'Anzahl6574Personen08', 'Anzahl7599Personen08',
       'Einpersonenhaushalt05', 'Mieterhaushalt05', 'Rentnerhaushalt05',
       'Jungerhaushalt05', 'FrauAlsReferenzperson05', 'MindestensEinAuto05',
       'MindestensEinVelo0

In [6]:
hh_data[['Grossregion01', 'Haushaltstyp14', 'Einkommensklasse08_151617']]

,Grossregion01,Haushaltstyp14,Einkommensklasse08_151617
HaushaltID,,,
360101,1,210,3
360102,1,110,2
360111,1,210,1
360122,1,300,2
360125,1,900,4
...,...,...,...
431723,7,900,3
431730,7,400,3
431738,7,300,2


Define filter categories and their values

In [7]:
filter_names = ['Grossregion01', 'Haushaltstyp14', 'Einkommensklasse08_151617']
filter_values = {}
for filter_name in filter_names:
    filter_values[filter_name] = hh_data[filter_name].unique()
filter_values

{'Grossregion01': array([1, 2, 3, 4, 5, 6, 7]),
 'Haushaltstyp14': array([210, 110, 300, 900, 230, 130, 400]),
 'Einkommensklasse08_151617': array([3, 2, 1, 4, 5])}

In [8]:
# Make a dictionrary with the filter values and their decription per filter category
category_names_dict = {}
for fn in filter_names:
    values = data_description.loc[data_description['Variablenname']==fn,'Modalität'].values
    description = data_description.loc[data_description['Variablenname']==fn,'Bezeichnung'].values
    category_names_dict[fn] = {value:descr for value,descr in zip(values, description)}

In [9]:
category_names_dict

{'Grossregion01': {1.0: 'Genferseeregion',
  2.0: 'Espace Mittelland',
  3.0: 'Nordwestschweiz',
  4.0: 'Zürich',
  5.0: 'Ostschweiz',
  6.0: 'Zentralschweiz',
  7.0: 'Tessin'},
 'Haushaltstyp14': {110.0: 'Einpersonenhaushalte unter 65',
  130.0: 'Einpersonenhaushalte ab 65',
  210.0: 'Paarhaushalte (beide unter 65) ohne weitere Haushaltsmitglieder',
  230.0: 'Paarhaushalte (mindestens eine Person ab 65) ohne weitere Haushaltsmitglieder',
  300.0: 'Einelternhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder',
  400.0: 'Paarhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder',
  900.0: 'Übrige Haushaltstypen'},
 'Einkommensklasse08_151617': {1.0: '1. Fünftel (< 4 914)',
  2.0: '2. Fünftel (4 914 – 7 264)',
  3.0: '3. Fünftel (7 265 – 9 990)',
  4.0: '4. Fünftel (9 991 – 13 621)',
  5.0: '5. Fünftel (≥ 13 622)'}}

In [49]:
import pickle

In [52]:
with open('./write_files/households_categories_names_dict.pickle', 'wb') as fh:
    pickle.dump(category_names_dict, fh)

In [94]:
category_names_dict_copy = category_names_dict.copy()
category_names_dict_copy['Einkommensklasse08_151617'][6.0] = 'Top 2% (>= 26 658)'
category_names_dict_copy['Einkommensklasse08_151617'][5.0] = '5. Fünftel (13 622 - 26 658)'
category_names_dict_copy

{'Grossregion01': {1.0: 'Genferseeregion',
  2.0: 'Espace Mittelland',
  3.0: 'Nordwestschweiz',
  4.0: 'Zürich',
  5.0: 'Ostschweiz',
  6.0: 'Zentralschweiz',
  7.0: 'Tessin'},
 'Haushaltstyp14': {110.0: 'Einpersonenhaushalte unter 65',
  130.0: 'Einpersonenhaushalte ab 65',
  210.0: 'Paarhaushalte (beide unter 65) ohne weitere Haushaltsmitglieder',
  230.0: 'Paarhaushalte (mindestens eine Person ab 65) ohne weitere Haushaltsmitglieder',
  300.0: 'Einelternhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder',
  400.0: 'Paarhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder',
  900.0: 'Übrige Haushaltstypen'},
 'Einkommensklasse08_151617': {1.0: '1. Fünftel (< 4 914)',
  2.0: '2. Fünftel (4 914 – 7 264)',
  3.0: '3. Fünftel (7 265 – 9 990)',
  4.0: '4. Fünftel (9 991 – 13 621)',
  5.0: '5. Fünftel (13 622 - 26 658)',
  6.0: 'Top 2% (>= 26 658)'}}

In [95]:
with open('./write_files/households_categories_names_dict_extended.pickle', 'wb') as fh:
    pickle.dump(category_names_dict_copy, fh)

We can on average make N_hh/min_sample_size number of catagories:

In [10]:
# N_hh/min_sample_size
print(9955/150)

66.36666666666666


This means we can combine each filter (Region, Income, hh type) (max 7*7=49)
But we need to check individual subgroups if they are larger than the min_sample_size

In [11]:
from colors import red, green, blue

In [12]:
combi_samplesize = {}
sample_too_small = 0
total_samples = 0
for combi in combinations(filter_names,2):
    total_rest_samples = 0
    number_of_rest_cats = 0
    total_nr_households = 0
    for i,j in product(filter_values[combi[0]],filter_values[combi[1]]):
        sample_size = len(hh_data.loc[(hh_data[combi[0]]==i)&(hh_data[combi[1]]==j)])
        # print(f'{combi[0]} : {i} , {combi[1]} : {j} , sample size : {sample_size}')
        combi_samplesize[combi] = sample_size
        total_nr_households += sample_size
        if sample_size < 150:
            sample_too_small+=1
            total_rest_samples += sample_size
            number_of_rest_cats += 1
            print(red(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
        else:
            print(green(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
        total_samples+=1
    print(f'size of rest {total_rest_samples} made up of {number_of_rest_cats} of categories')
    print(f'total number of households {total_nr_households}')
print(f'{sample_too_small} samples out of {total_samples} too small')

Grossregion01 : Genferseeregion , Haushaltstyp14 : Paarhaushalte (beide unter 65) ohne weitere Haushaltsmitglieder , sample size : 264
Grossregion01 : Genferseeregion , Haushaltstyp14 : Einpersonenhaushalte unter 65 , sample size : 336
Grossregion01 : Genferseeregion , Haushaltstyp14 : Einelternhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder , sample size : 117
Grossregion01 : Genferseeregion , Haushaltstyp14 : Übrige Haushaltstypen , sample size : 63
Grossregion01 : Genferseeregion , Haushaltstyp14 : Paarhaushalte (mindestens eine Person ab 65) ohne weitere Haushaltsmitglieder , sample size : 223
Grossregion01 : Genferseeregion , Haushaltstyp14 : Einpersonenhaushalte ab 65 , sample size : 163
Grossregion01 : Genferseeregion , Haushaltstyp14 : Paarhaushalte mit Kindern (mindestens eines unter 25) ohne weitere Haushaltsmitglieder , sample size : 426
Grossregion01 : Espace Mittelland , Haushaltstyp14 : Paarhaushalte (beide unter 65) ohne weitere Haushal

Ok,so for each combination of keys (region, houshold type and income group), we'll only use the ones with a sample greater than 150 households, and aggregate the rest into a "rest group". Only Tessin einkommensklasse 5 (>= 13 622) would be alone in the rest group with 139 housholds. 



In [13]:
combi_samplesize

{('Grossregion01', 'Haushaltstyp14'): 301,
 ('Grossregion01', 'Einkommensklasse08_151617'): 139,
 ('Haushaltstyp14', 'Einkommensklasse08_151617'): 973}

In [14]:
# make category label dict
filter_labels = {'Grossregion01': {
  1.0: 'A',
  2.0: 'B',
  3.0: 'C',
  4.0: 'D',
  5.0: 'E',
  6.0: 'F',
  7.0: 'G'},
 'Haushaltstyp14': {
  110.0: 'a',
  130.0: 'b',
  210.0: 'c',
  230.0: 'd',
  300.0: 'e',
  400.0: 'f',
  900.0: 'g'},
 'Einkommensklasse08_151617': {
  1.0: '1',
  2.0: '2',
  3.0: '3',
  4.0: '4',
  5.0: '5'}
  }
filter_labels

{'Grossregion01': {1.0: 'A',
  2.0: 'B',
  3.0: 'C',
  4.0: 'D',
  5.0: 'E',
  6.0: 'F',
  7.0: 'G'},
 'Haushaltstyp14': {110.0: 'a',
  130.0: 'b',
  210.0: 'c',
  230.0: 'd',
  300.0: 'e',
  400.0: 'f',
  900.0: 'g'},
 'Einkommensklasse08_151617': {1.0: '1',
  2.0: '2',
  3.0: '3',
  4.0: '4',
  5.0: '5'}}

In [53]:
# write to pickle
with open('./write_files/households_categories_labels_dict.pickle', 'wb') as fh:
    pickle.dump(filter_labels, fh)

## Use above categories to create a clustering dataframe

In [41]:
%%time
sample_too_small = 0
total_samples = 0
# Make one big dataframe (each household will be in there 3 times, for the 3 different axes combinations)
clustering_df_grande = pd.DataFrame(columns=['cluster_code', 'n_households', 'n_people', 'gross_income'])
# Loop through the different axes combinations
for combi in combinations(filter_names,2):
    print(combi)
    clustering_df = pd.DataFrame(index=hh_data.index, columns=['cluster_code', 'n_households', 'n_people', 'gross_income'])
    rest_indices = []
    total_rest_samples = 0
    # Loop through the different value combinations
    for i,j in product(filter_values[combi[0]],filter_values[combi[1]]):
        subset_indices = hh_data.loc[(hh_data[combi[0]]==i)&(hh_data[combi[1]]==j)].index
        sample_size = len(subset_indices)
        # print(f'{combi[0]} : {i} , {combi[1]} : {j} , sample size : {sample_size}'
        # If a sample is too small in size, it will go into the rest group for the axes combination
        if sample_size < 150:
            [rest_indices.append(i) for i in subset_indices]
            total_rest_samples += sample_size
            # print(red(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
        else:
            # print(green(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
            cluster_code = f'{filter_labels[combi[0]][i]}{filter_labels[combi[1]][j]}'
            print(cluster_code)
            clustering_df.loc[subset_indices,'cluster_code'] = cluster_code
            clustering_df.loc[subset_indices,'n_households'] = sample_size
            clustering_df.loc[subset_indices,'n_people'] = hh_data.loc[subset_indices,'AnzahlPersonen98']
            clustering_df.loc[subset_indices,'gross_income'] = hh_data.loc[subset_indices,'Bruttoeinkommen08']
    # Now make a cat cor the rest group
    cluster_code = f'{combi[0][:3]}_{combi[1][:3]}_rest'
    print(cluster_code)
    clustering_df.loc[rest_indices,'cluster_code'] = cluster_code
    clustering_df.loc[rest_indices,'n_households'] = total_rest_samples
    clustering_df.loc[rest_indices,'n_people'] = hh_data.loc[rest_indices,'AnzahlPersonen98']
    clustering_df.loc[rest_indices,'gross_income'] = hh_data.loc[rest_indices,'Bruttoeinkommen08']
    clustering_df_grande = pd.concat([clustering_df_grande, clustering_df], axis=0)


('Grossregion01', 'Haushaltstyp14')
Ac
Aa
Ad
Ab
Af
Bc
Ba
Bd
Bb
Bf
Cc
Ca
Cd
Cf
Dc
Da
Dd
Db
Df
Ec
Ea
Ed
Ef
Fc
Fa
Fd
Ff
Gc
Ga
Gd
Gf
Gro_Hau_rest
('Grossregion01', 'Einkommensklasse08_151617')
A3
A2
A1
A4
A5
B3
B2
B1
B4
B5
C3
C2
C1
C4
C5
D3
D2
D1
D4
D5
E3
E2
E1
E4
E5
F3
F2
F1
F4
F5
G3
G2
G1
G4
Gro_Ein_rest
('Haushaltstyp14', 'Einkommensklasse08_151617')
c3
c2
c4
c5
a3
a2
a1
a4
d3
d2
d1
d4
b2
b1
f3
f2
f4
f5
Hau_Ein_rest
CPU times: user 268 ms, sys: 6.09 ms, total: 274 ms
Wall time: 272 ms


In [42]:
clustering_df_grande.index.name = 'HaushaltID'
clustering_df_grande

,cluster_code,n_households,n_people,gross_income
HaushaltID,,,,
360101,Ac,264,2,8275.2175
360102,Aa,336,1,5213.016667
360111,Ac,264,2,1005.570833
360122,Gro_Hau_rest,1267,2,7211.65121
360125,Gro_Hau_rest,1267,2,11201.230914
...,...,...,...,...
431723,Hau_Ein_rest,1257,2,8113.475806
431730,f3,543,3,9526.701613
431738,Hau_Ein_rest,1257,4,5291.889913


In [54]:
len(clustering_df_grande['cluster_code'].unique())

86

In [43]:
# Now save clustering dataframe
fp_clustering = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering.csv'
clustering_df_grande.to_csv(fp_clustering)

## Simple clustering

Now let's also do a simple clustering, where we only group along 1 axis at a time


In [76]:
# first we add another income group for the top 2%
hh_data_copy = hh_data.copy()
hh_data_copy.loc[hh_data_copy['Bruttoeinkommen08'] >= np.percentile(hh_data_copy['Bruttoeinkommen08'], 98), 'Einkommensklasse08_151617'] = 6


In [78]:
filter_values_copy = filter_values.copy()
filter_values_copy['Einkommensklasse08_151617'] = np.arange(6)+1
filter_values_copy

{'Grossregion01': array([1, 2, 3, 4, 5, 6, 7]),
 'Haushaltstyp14': array([210, 110, 300, 900, 230, 130, 400]),
 'Einkommensklasse08_151617': array([1, 2, 3, 4, 5, 6])}

In [84]:
filter_labels_copy = filter_labels
filter_labels_copy['Einkommensklasse08_151617'][6.0] = '6'
filter_labels_copy

{'Grossregion01': {1.0: 'A',
  2.0: 'B',
  3.0: 'C',
  4.0: 'D',
  5.0: 'E',
  6.0: 'F',
  7.0: 'G'},
 'Haushaltstyp14': {110.0: 'a',
  130.0: 'b',
  210.0: 'c',
  230.0: 'd',
  300.0: 'e',
  400.0: 'f',
  900.0: 'g'},
 'Einkommensklasse08_151617': {1.0: '1',
  2.0: '2',
  3.0: '3',
  4.0: '4',
  5.0: '5',
  6.0: '6'}}

In [91]:
# write to pickle
with open('./write_files/households_categories_labels_dict_extended.pickle', 'wb') as fh:
    pickle.dump(filter_labels_copy, fh)

In [85]:
%%time
sample_too_small = 0
total_samples = 0
# Make one big dataframe (each household will be in there 3 times, for the 3 different axes combinations)
clustering_df_grande_simple = pd.DataFrame(columns=['cluster_code', 'n_households', 'n_people', 'gross_income'])
# Loop through the different axes combinations
for filter_name in filter_names:
    print(filter_name)
    clustering_df = pd.DataFrame(index=hh_data_copy.index, columns=['cluster_code', 'n_households', 'n_people', 'gross_income'])
    rest_indices = []
    total_rest_samples = 0
    # Loop through the different value combinations
    for i in filter_values_copy[filter_name]:      
        subset_indices = hh_data_copy.loc[(hh_data_copy[filter_name]==i)].index
        sample_size = len(subset_indices)
        # print(f'{combi[0]} : {i} , {combi[1]} : {j} , sample size : {sample_size}'
        # If a sample is too small in size, it will go into the rest group for the axes combination
        if sample_size < 150:
            [rest_indices.append(i) for i in subset_indices]
            total_rest_samples += sample_size
            # print(red(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
        else:
            # print(green(f'{combi[0]} : {category_names_dict[combi[0]][i]} , {combi[1]} : {category_names_dict[combi[1]][j]} , sample size : {sample_size}'))
            cluster_code = f'{filter_labels_copy[filter_name][i]}'
            print(cluster_code)
            clustering_df.loc[subset_indices,'cluster_code'] = cluster_code
            clustering_df.loc[subset_indices,'n_households'] = sample_size
            clustering_df.loc[subset_indices,'n_people'] = hh_data_copy.loc[subset_indices,'AnzahlPersonen98']
            clustering_df.loc[subset_indices,'gross_income'] = hh_data_copy.loc[subset_indices,'Bruttoeinkommen08']
    # Now make a cat cor the rest group
    if len(rest_indices)>0:
        cluster_code = f'{filter_name[:3]}_rest'
        print(cluster_code)
        clustering_df.loc[rest_indices,'cluster_code'] = cluster_code
        clustering_df.loc[rest_indices,'n_households'] = total_rest_samples
        clustering_df.loc[rest_indices,'n_people'] = hh_data_copy.loc[rest_indices,'AnzahlPersonen98']
        clustering_df.loc[rest_indices,'gross_income'] = hh_data_copy.loc[rest_indices,'Bruttoeinkommen08']
    clustering_df_grande_simple = pd.concat([clustering_df_grande_simple, clustering_df], axis=0)


Grossregion01
A
B
C
D
E
F
G
Haushaltstyp14
c
a
e
g
d
b
f
Einkommensklasse08_151617
1
2
3
4
5
6
CPU times: user 63.5 ms, sys: 3.25 ms, total: 66.7 ms
Wall time: 66 ms


In [86]:
clustering_df_grande_simple.index.name = 'HaushaltID'
clustering_df_grande_simple

,cluster_code,n_households,n_people,gross_income
HaushaltID,,,,
360101,A,1592,2,8275.2175
360102,A,1592,1,5213.016667
360111,A,1592,2,1005.570833
360122,A,1592,2,7211.65121
360125,A,1592,2,11201.230914
...,...,...,...,...
431723,3,2057,2,8113.475806
431730,3,2057,3,9526.701613
431738,2,2017,4,5291.889913


In [87]:
# Now save clustering dataframe
fp_clustering_simple = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering_simple.csv'
clustering_df_grande_simple.to_csv(fp_clustering_simple)

In [47]:
# Now check that we can indeed read this again
test = pd.read_csv(fp_clustering, index_col=0)
test

,cluster_code,n_households,n_people,gross_income
HaushaltID,,,,
360101,Ac,264,2,8275.217500
360102,Aa,336,1,5213.016667
360111,Ac,264,2,1005.570833
360122,Gro_Hau_rest,1267,2,7211.651210
360125,Gro_Hau_rest,1267,2,11201.230914
...,...,...,...,...
431723,Hau_Ein_rest,1257,2,8113.475806
431730,f3,543,3,9526.701613
431738,Hau_Ein_rest,1257,4,5291.889913


In [48]:
# Now check that we can indeed read this again
test_simple = pd.read_csv(fp_clustering_simple, index_col=0)
test_simple

,cluster_code,n_households,n_people,gross_income
HaushaltID,,,,
360101,A,1592,2,8275.217500
360102,A,1592,1,5213.016667
360111,A,1592,2,1005.570833
360122,A,1592,2,7211.651210
360125,A,1592,2,11201.230914
...,...,...,...,...
431723,3,2057,2,8113.475806
431730,3,2057,3,9526.701613
431738,2,2017,4,5291.889913


# Now combine the csv's into one file

In [ ]:
# read in clustering data. (append combined and simple clustering
# combined clustering
fp_habe_clustering = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering.csv'
hh_clusters = pd.read_csv(fp_habe_clustering, index_col=0)
hh_clusters.reset_index(inplace=True)  # reset the index so to keep 'HaushaltID'
hh_clusters.set_index('cluster_code', inplace=True)  # set index to cluster code/label
# simple clustering
fp_habe_clustering = '/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering_simple.csv'
hh_clusters_s = pd.read_csv(fp_habe_clustering, index_col=0)
hh_clusters_s.reset_index(inplace=True)  # reset the index so to keep 'HaushaltID'
hh_clusters_s.set_index('cluster_code', inplace=True)  # set index to cluster code/label

hh_clusters = pd.concat((hh_clusters,hh_clusters_s))
hh_clusters.to_csv('/Users/ajakobs/Documents/CCL/scripts/notebooks/write_files/ccl_dev/households_clustering_total.csv')
